## Preliminaries

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pickle
import os
from collections import Counter
import pandas as pd

* Table of Contents                               
{:toc}      

# Datasets
**HDFS**  
Open dataset인 [Hadoop Distributed File System(HDFS)](https://github.com/logpai/loghub/blob/master/HDFS/HDFS_2k.log) 데이터셋은 하둡 분산처리 시스템의 로그로 이루어져 있습니다.  
HDFS 데이터셋은 정상 시스템 로그와 이상 시스템 로그로 이루어져 있습니다.  
로그 내의 **block_id**는 해당 로그가 발생한 사건의 고유 정보를 의미합니다.     
같은 **block_id** 데이터셋은 모두 같은 사건에 대한 정보를 담고 있습니다.  
따라서 **block_id** 데이터를 묶어 같은 **block_id** 이루어진 로그 시퀀스는 한 개의 로그 시퀀스로 판단합니다.  

**Log Parser**  
Log Parser는 비정형 로그 데이터를 정형화된 템플릿으로 변환하는 역할을 수행합니다.  
본 데이터셋에 대해서는 [Drain Log Parser](https://github.com/logpai/logparser/tree/master/logparser/Drain)를 이용합니다.  
데이터에 대해 Parser를 학습하면 정형화된 템플릿 규칙을 만들어냅니다.   
예시를 들어보겠습니다. 아래는 HDFS 데이터셋에 존재하는 기존 로그 파일입니다.  

아래는 Drain Log Parser를 HDFS 데이터셋에 대해서 학습을 시켜 추출한 템플릿입니다.  

In [15]:
templates = pd.read_csv('./data/train_templates.csv')
templates.head()

,EventId,EventTemplate,Occurrences
0,09a53393,Receiving block <*> src: <*> dest: <*>,1341924
1,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,446578
2,dc2c74b7,PacketResponder <*> for block <*> terminating,1339734
3,e3df2680,Received block <*> of size <*> from <*>,1339734
4,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,1343739


해당 템플릿에 따르면 위의 데이터는  


로 변하게 됩니다.

본 튜토리얼에서는 Log Parser에 대한 자세한 학습 과정 및 설명은 생략합니다.  
data 폴더 내의 train.pkl과 test.pkl은 학습된 parser를 통해 전처리가 진행된 파일입니다.  

{block_id : log sequence} 형식의 dictionary입니다. 예시는 아래와 같습니다.

In [16]:
with open('./data/train.pkl', 'rb') as f:
    train = pickle.load(f)
with open('./data/test.pkl', 'rb') as f:
    test = pickle.load(f)    

In [19]:
list(test.items())[:2]

[('blk_-1000195927844309648',
  [0, 0, 0, 3, 17, 18, 17, 18, 17, 18, 6, 6, 6, 14, 14, 14, 16, 16, 16]),
 ('blk_-1000245396392748444',
  [3, 0, 0, 0, 6, 6, 6, 17, 18, 17, 18, 17, 18, 2, 14, 14, 14, 16, 16, 16])]

# Data Encoding
데이터를 모델에 넣기 위해서는 인코딩 과정이 필요합니다.  
본 과정에서는 이를 위해 간단한 Vocab 모듈을 제작해 사용합니다.  
Vocab 모듈은 input 데이터의 각 원소들을 해당 원소에 해당하는 embedding matrix에 매칭시키는 역할을 수행합니다.  

In [22]:
class Vocab:
    ''' 데이터 인코딩에 필요한 vocab을 만드는 모듈 '''
    def __init__(self, vocab_path):
        # 모델에 필요한 special token을 정의합니다.
        self.special_tokens = ['<PAD>', '<BOS>', '<EOS>', '<UNK>']
        self.vocab_path = vocab_path
        self.data = None
        self.vocab = None
        if os.path.isfile(vocab_path):
            self.load_vocab()

    def load_data(self, data_path):
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
        self.data = list(data.values())

    def create_vocab(self):
        assert self.data is not None
        ctr = Counter()
        for d in self.data:
            ctr.update(d)
        vocab = list(ctr.keys())
        self.vocab = self.special_tokens + vocab
        self.save_vocab()

    def load_vocab(self):
        with open(self.vocab_path, 'r') as f:
            vocab = []
            for line in f:
                try:
                    vocab.append(int(line.strip()))
                except:
                    vocab.append(line.strip())
        self.vocab = vocab

    def save_vocab(self):
        assert self.vocab is not None
        with open(self.vocab_path, 'w') as f:
            for word in self.vocab:
                f.write(str(word) + '\n')

In [25]:
data_path = './data/train.pkl'  # vocab 파일을 제작할 파일 (train set)
vocab_path = './data/vocab.txt'  # 만들어진 vocab 파일의 위치  

# Create vocab.txt
vocab_module = Vocab(vocab_path)
vocab_module.load_data(data_path)
vocab_module.create_vocab()
log_vocab = vocab_module.vocab

In [26]:
log_vocab

['<PAD>',
 '<BOS>',
 '<EOS>',
 '<UNK>',
 0,
 3,
 17,
 18,
 6,
 11,
 7,
 14,
 16,
 2,
 8,
 4,
 9,
 15,
 5,
 10,
 13,
 1,
 12]

### Pytorch Custom Datasets
파이토치는 직접 datasets 모듈을 만들어 사용할 수 있습니다.  
이 때 필요한 것은 해당 모듈에  
\__len__  
\__getitem__  
메소드를 오버라이딩해어야 한다는 것입니다.  


In [27]:
class LogLoader(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.blk_id, self.data, self.lengths = self.data_load()
        # Special tokens 정의
        self.pad, self.bos, self.eos, self.unk = 0, 1, 2, 3

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        input_log, output_log = self.preprocess(self.data[item])
        return input_log, output_log, self.lengths[item]

    def data_load(self):
        with open(self.data_path, 'rb') as f:
            data = pickle.load(f)
        blk, logs = list(zip(*data.items()))
        lengths = [len(d) + 1 for d in logs]
        return blk, logs, lengths

    def preprocess(self, log):
        # Special token 추가를 위해 input 조정
        log = [int(l) + 4 for l in log]
        # eos, bos 추가
        logs = (log + [self.eos], [self.bos] + log)
        return logs

    def padding(self, log, max_len):
        return [l + [self.pad] * (max_len - len(l)) for l in log]

    def batch_sequence(self, batch):
        input_log, output_log, lengths = list(zip(*batch))
        assert len(input_log) == len(output_log)
        assert len(input_log) == len(lengths)

        max_seq_length = max(lengths)

        # pad tokens
        input_log, output_log = [self.padding(log, max_seq_length) for log in (input_log, output_log)]
        assert len(output_log[0]) == max_seq_length

        return (torch.tensor(d) for d in (input_log, output_log, lengths))


In [8]:
type(log)

dict

In [9]:
list(log.items())[0]

('blk_-1000002529962039464', [0, 0, 0, 3, 17, 18, 17, 18, 6, 6, 6, 17, 18])

In [3]:
templates.head()

,EventId,EventTemplate,Occurrences
0,09a53393,Receiving block <*> src: <*> dest: <*>,381308
1,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,128483
2,dc2c74b7,PacketResponder <*> for block <*> terminating,366945
3,e3df2680,Received block <*> of size <*> from <*>,366780
4,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,376002


## Simple Pytorch Explanations

## Datasets

In [8]:
logs = list(zip(*log.items()))

In [11]:
logs[1][0]

[0, 0, 0, 3, 17, 18, 17, 18, 6, 6, 6, 17, 18]

In [13]:
'test'.astype(int)

AttributeError: 'str' object has no attribute 'astype'